In [156]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [157]:
#Параметры
TARGET_STORES = [15292, 15618, 15681, 15686, 15755, 16752, 17646]
TARGET_PRODUCTS = [19345, 19347, 20835, 21159, 24849, 25023, 34570, 36843, 38118, 38119, 59996, 61604, 65034, 65042, 65043, 65114]
MASLENITSA_2024_START = datetime(2024, 3, 11)
MASLENITSA_2024_END = datetime(2024, 3, 17)
SHELF_LIFE = 5  #срок годности 5 дней
DISCOUNT_DAY = 5  #день, когда товар продают по скидке
DISCOUNT_RATE = 0.4  #размер скидки - 40%

In [158]:
#Загружаем данные
df = pd.read_csv("/content/данные.csv", delimiter = ";")
df

,2022-01-01,15292,2021-06-12,РЦ_1,Блины,19345,0.000,0.000.1,0.000.2,"31,3041095733643",108.000000
0,2022-01-01,15618,2021-07-14,РЦ_1,Блины,19345,1.0,0.0,0.0,0,108.0
1,2022-01-01,15755,2021-08-14,РЦ_2,Блины,19345,5.0,5.0,1.0,"9,79758167266846",108.0
2,2022-01-01,15292,2021-06-12,РЦ_1,Блины,19347,2.0,0.0,0.0,0,186.0
3,2022-01-01,15681,2021-08-13,РЦ_3,Блины,19347,0.0,0.0,1.0,0,186.0
4,2022-01-01,15686,2021-08-30,РЦ_3,Блины,19347,3.0,3.0,1.0,"0,821054041385651",186.0
...,...,...,...,...,...,...,...,...,...,...,...
35987,2024-01-31,16752,2022-07-18,РЦ_4,Блины,65043,3.0,0.0,0.0,0,114.0
35988,2024-01-31,15618,2021-07-14,РЦ_1,Блины,65114,0.0,0.0,0.0,0,228.0
35989,2024-01-31,15681,2021-08-13,РЦ_3,Блины,65114,5.0,0.0,0.0,0,228.0
35990,2024-01-31,15686,2021-08-30,РЦ_3,Блины,65114,6.0,0.0,0.0,"2,04888105392456",228.0


In [159]:
#Создаем имена для столбцов
df.columns = ['Дата', 'id_точки', 'Дата_открытия', 'РЦ', 'Категория', 'id_товара',
              'Продажи_шт', 'Продажи_с_дисконтом', 'Списания', 'Дефицит', 'Цена']
df

,Дата,id_точки,Дата_открытия,РЦ,Категория,id_товара,Продажи_шт,Продажи_с_дисконтом,Списания,Дефицит,Цена
0,2022-01-01,15618,2021-07-14,РЦ_1,Блины,19345,1.0,0.0,0.0,0,108.0
1,2022-01-01,15755,2021-08-14,РЦ_2,Блины,19345,5.0,5.0,1.0,"9,79758167266846",108.0
2,2022-01-01,15292,2021-06-12,РЦ_1,Блины,19347,2.0,0.0,0.0,0,186.0
3,2022-01-01,15681,2021-08-13,РЦ_3,Блины,19347,0.0,0.0,1.0,0,186.0
4,2022-01-01,15686,2021-08-30,РЦ_3,Блины,19347,3.0,3.0,1.0,"0,821054041385651",186.0
...,...,...,...,...,...,...,...,...,...,...,...
35987,2024-01-31,16752,2022-07-18,РЦ_4,Блины,65043,3.0,0.0,0.0,0,114.0
35988,2024-01-31,15618,2021-07-14,РЦ_1,Блины,65114,0.0,0.0,0.0,0,228.0
35989,2024-01-31,15681,2021-08-13,РЦ_3,Блины,65114,5.0,0.0,0.0,0,228.0
35990,2024-01-31,15686,2021-08-30,РЦ_3,Блины,65114,6.0,0.0,0.0,"2,04888105392456",228.0


In [160]:
# Преобразование дат
df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d', errors='coerce')
df['Дата_открытия'] = pd.to_datetime(df['Дата_открытия'], format='%Y-%m-%d', errors='coerce')

In [161]:
#Фильтруем по нужным товарам и точкам
df_filtered = df[(df['id_товара'].isin(TARGET_PRODUCTS)) &
                 (df['id_точки'].isin(TARGET_STORES))].copy()

print(f"Итого {len(df_filtered)} строк")

Итого 35882 строк


In [162]:
#Добавляем даты предыдущих Маслениц
def get_maslenitsa_dates(year):
    maslenitsa_dates = {
        2022: (datetime(2022, 2, 28), datetime(2022, 3, 6)),
        2023: (datetime(2023, 2, 20), datetime(2023, 2, 26)),
        2024: (datetime(2024, 3, 11), datetime(2024, 3, 17))
    }
    return maslenitsa_dates.get(year, (None, None))

In [163]:
# Добавление признаков
df_filtered['Год'] = df_filtered['Дата'].dt.year
df_filtered['Месяц'] = df_filtered['Дата'].dt.month
df_filtered['День_недели'] = df_filtered['Дата'].dt.dayofweek
df_filtered['Неделя_года'] = df_filtered['Дата'].dt.isocalendar().week
df_filtered['День_месяца'] = df_filtered['Дата'].dt.day

In [164]:
# Определение Масленицы в исторических данных
df_filtered['Масленица'] = False
for year in [2022, 2023]:
    start, end = get_maslenitsa_dates(year)
    if start and end:
        df_filtered.loc[(df_filtered['Дата'] >= start) & (df_filtered['Дата'] <= end), 'Масленица'] = True
maslenitsa_data = df_filtered[df_filtered['Масленица'] == True].copy()
regular_data = df_filtered[df_filtered['Масленица'] == False].copy()

In [165]:
maslenitsa_data

,Дата,id_точки,Дата_открытия,РЦ,Категория,id_товара,Продажи_шт,Продажи_с_дисконтом,Списания,Дефицит,Цена,Год,Месяц,День_недели,Неделя_года,День_месяца,Масленица
2080,2022-02-28,15292,2021-06-12,РЦ_1,Блины,19345,139.0,0.0,0.0,0,108.0,2022,2,0,9,28,True
2081,2022-02-28,15618,2021-07-14,РЦ_1,Блины,19345,8.0,0.0,0.0,"3,63942360877991",108.0,2022,2,0,9,28,True
2082,2022-02-28,15755,2021-08-14,РЦ_2,Блины,19345,42.0,0.0,0.0,0,108.0,2022,2,0,9,28,True
2083,2022-02-28,15292,2021-06-12,РЦ_1,Блины,19347,43.0,0.0,0.0,0,186.0,2022,2,0,9,28,True
2084,2022-02-28,15618,2021-07-14,РЦ_1,Блины,19347,2.0,0.0,0.0,0,186.0,2022,2,0,9,28,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16376,2023-02-26,15681,2021-08-13,РЦ_3,Блины,65042,8.0,0.0,0.0,0,178.0,2023,2,6,8,26,True
16377,2023-02-26,15686,2021-08-30,РЦ_3,Блины,65042,3.0,0.0,0.0,0,178.0,2023,2,6,8,26,True
16378,2023-02-26,15292,2021-06-12,РЦ_1,Блины,65043,2.0,0.0,0.0,"24,875",114.0,2023,2,6,8,26,True
16379,2023-02-26,15681,2021-08-13,РЦ_3,Блины,65043,38.0,0.0,0.0,"11,0172643661499",114.0,2023,2,6,8,26,True


In [166]:
#Вычисляем средние значения
maslenitsa_mean = pd.to_numeric(maslenitsa_data['Продажи_шт']).fillna(0).mean() #средние продажи в дни Масленицы
regular_mean = pd.to_numeric(regular_data['Продажи_шт']).fillna(0).mean() #средние продажи в обычные дни

print(f"Средние продажи во время Масленицы: {maslenitsa_mean:.2f} шт/день")
print(f"Средние продажи в обычные дни: {regular_mean:.2f} шт/день")

#Коэффициента роста
growth_factor = maslenitsa_mean/regular_mean
print(f"Коэффициент роста продаж во время Масленицы: {growth_factor:.2f}x")

Средние продажи во время Масленицы: 11.21 шт/день
Средние продажи в обычные дни: 6.37 шт/день
Коэффициент роста продаж во время Масленицы: 1.76x


In [167]:
def forecast_sales(store_id, product_id, forecast_dates):

    #Данные для конкретной точки и товара
    store_product_data = df_filtered[
        (df_filtered['id_точки'] == store_id) &
        (df_filtered['id_товара'] == product_id)
    ].copy()

    if len(store_product_data) == 0:
        return np.zeros(len(forecast_dates))

    forecasts = []

    for forecast_date in forecast_dates:
        day_of_week = forecast_date.dayofweek

        #Базовый прогноз: среднее за аналогичный день недели
        same_weekday_data = store_product_data[store_product_data['Дата'].dt.dayofweek == day_of_week]
        if len(same_weekday_data) > 0:
            base_forecast = same_weekday_data['Продажи_шт'].mean()
            if pd.isna(base_forecast) or base_forecast == 0:
                base_forecast = store_product_data['Продажи_шт'].mean()
                if pd.isna(base_forecast):
                    base_forecast = 0
        else:
            base_forecast = store_product_data['Продажи_шт'].mean()
            if pd.isna(base_forecast):
                base_forecast = 0

        #Учет эффекта Масленицы
        maslenitsa_factor = 1.0
        if len(maslenitsa_data) > 0:
            same_weekday_maslenitsa = maslenitsa_data[
                (maslenitsa_data['id_точки'] == store_id) &
                (maslenitsa_data['id_товара'] == product_id) &
                (maslenitsa_data['Дата'].dt.dayofweek == day_of_week)
            ]
            if len(same_weekday_maslenitsa) > 0:
                maslenitsa_sales = same_weekday_maslenitsa['Продажи_шт'].mean()
                if not pd.isna(maslenitsa_sales) and maslenitsa_sales > 0:
                    regular_same_weekday = regular_data[
                        (regular_data['id_точки'] == store_id) &
                        (regular_data['id_товара'] == product_id) &
                        (regular_data['Дата'].dt.dayofweek == day_of_week)
                    ]
                    if len(regular_same_weekday) > 0:
                        regular_sales = regular_same_weekday['Продажи_шт'].mean()
                        if not pd.isna(regular_sales) and regular_sales > 0:
                            maslenitsa_factor = maslenitsa_sales / regular_sales
                        else:
                            maslenitsa_factor = 1.5  #Дефолтный коэффициент
                    else:
                        maslenitsa_factor = 1.5  #Дефолтный коэффициент
                else:
                    maslenitsa_factor = 1.5  #Дефолтный коэффициент

        #Учет тренда за последние 3 месяца
        recent_data = store_product_data[store_product_data['Дата'] >= (forecast_date - timedelta(days=90))]
        if len(recent_data) > 0:
            recent_mean = recent_data['Продажи_шт'].mean()
            if not pd.isna(recent_mean) and base_forecast > 0:
                trend_factor = recent_mean / base_forecast
                if not pd.isna(trend_factor) and trend_factor > 0:
                    base_forecast = base_forecast * (0.5 + 0.5 * trend_factor)  #Взвешенное среднее

        forecast = base_forecast * maslenitsa_factor

        #Проверка на NaN и бесконечность
        if pd.isna(forecast) or np.isinf(forecast):
            forecast = 0

        #Округление до целого, минимум 0
        forecasts.append(max(0, round(forecast)))

    return np.array(forecasts)

In [168]:
# Генерация дат прогноза
forecast_dates = pd.date_range(start=MASLENITSA_2024_START, end=MASLENITSA_2024_END, freq='D')

# Создание прогноза
forecasts_list = []
for store_id in TARGET_STORES:
    for product_id in TARGET_PRODUCTS:
        forecast_values = forecast_sales(store_id, product_id, forecast_dates)
        for i, date in enumerate(forecast_dates):
            forecasts_list.append({
                'Дата': date,
                'id_точки': store_id,
                'id_товара': product_id,
                'Прогноз_продаж': forecast_values[i]
            })

forecast_df = pd.DataFrame(forecasts_list)
print(f"Суммарный прогноз продаж: {forecast_df['Прогноз_продаж'].sum():.0f} шт")

Суммарный прогноз продаж: 5720 шт


In [169]:
#Расчет объема заказа
def calculate_order_quantity(forecast_df, store_id, product_id):
    store_product_forecast = forecast_df[
        (forecast_df['id_точки'] == store_id) &
        (forecast_df['id_товара'] == product_id)
    ].sort_values('Дата').copy()

    if len(store_product_forecast) == 0:
        return pd.DataFrame()

    orders_list = []
    #остатки по возрасту: ключ - возраст в днях (0-3), значение - количество
    #Товар возраста 4 в конце дня станет возрастом 5 и будет продан со скидкой
    stock_by_age = {0: 0, 1: 0, 2: 0, 3: 0}

    #сброс индекса для удобства работы с порядковыми номерами
    store_product_forecast = store_product_forecast.reset_index(drop=True)

    for day_idx, row in store_product_forecast.iterrows():
        date = row['Дата']
        forecast_sales = row['Прогноз_продаж']

        day5_stock = stock_by_age[3]
        for age in range(3, 0, -1):
            stock_by_age[age] = stock_by_age[age - 1]
        stock_by_age[0] = 0

        #Расчет необходимого заказа ДО продаж
        # Учитываем остатки перед заказом (без товара 5-го дня, т.к. он будет продан или списан)
        available_stock_before_order = sum(stock_by_age.values())

        #Учитываем будущий спрос (следующие дни в пределах срока годности)
        future_demand = 0
        if day_idx < len(store_product_forecast) - 1:
            # Средний прогноз на следующие 4 дня (в пределах срока годности)
            next_days = min(3, len(store_product_forecast) - day_idx - 1)
            if next_days > 0:
                future_forecasts = store_product_forecast.loc[day_idx+1:day_idx+next_days, 'Прогноз_продаж']
                future_demand = future_forecasts.mean()

        #Стратегия заказа: покрываем текущий прогноз + 30% от среднего будущего спроса
        #с учетом того, что товар должен быть свежим
        safety_factor = 1.15  # 15% страховой запас
        target_stock = max(forecast_sales, (forecast_sales + future_demand * 0.3) * safety_factor)
        order_quantity = max(0, int(np.ceil(target_stock - available_stock_before_order)))

        #Добавляем заказ в остатки (новый товар, возраст 0)
        stock_by_age[0] = order_quantity

        #Продажи
        #Сначала продаем товар 5-го дня (со скидкой 40%)
        discounted_sales = min(day5_stock, forecast_sales)
        remaining_demand = forecast_sales - discounted_sales

        #Затем продаем остальной товар (от свежего к старому)
        regular_sales = 0
        for age in range(3):
            if remaining_demand <= 0:
                break
            sales_from_age = min(stock_by_age[age], remaining_demand)
            stock_by_age[age] -= sales_from_age
            regular_sales += sales_from_age
            remaining_demand -= sales_from_age

        #Дефицит
        deficit = max(0, remaining_demand)

        #Товар 5-го дня, который не был продан, списывается
        expired = day5_stock - discounted_sales

        #Итоговый остаток после продаж
        final_stock = sum(stock_by_age.values())

        orders_list.append({
            'Дата': date,
            'id_точки': store_id,
            'id_товара': product_id,
            'Прогноз_продаж': forecast_sales,
            'Объем_заказа': order_quantity,
            'Продажи_обычные': regular_sales,
            'Продажи_со_скидкой': discounted_sales,
            'Списания': expired,
            'Остаток_конец_дня': final_stock,
            'Дефицит': deficit
        })

    return pd.DataFrame(orders_list)

In [170]:
#Расчет заказов для всех комбинаций
all_orders = []
for store_id in TARGET_STORES:
    for product_id in TARGET_PRODUCTS:
        orders = calculate_order_quantity(forecast_df, store_id, product_id)
        if len(orders) > 0:
            all_orders.append(orders)

orders_df = pd.concat(all_orders, ignore_index=True)

In [171]:
#Сохранение результатов
forecast_df.to_csv('прогноз_продаж_масленица_2024.csv', index=False, encoding='utf-8-sig')
orders_df.to_csv('объем_заказа_масленица_2024.csv', index=False, encoding='utf-8-sig')
forecast_df.to_excel('forecast_data.xlsx', index=False)
orders_df.to_excel('orders_data.xlsx', index=False)

In [172]:
# Визуализация

# 1. Общий прогноз продаж по дням Масленицы
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1.1. Прогноз продаж по дням
daily_forecast = forecast_df.groupby('Дата')['Прогноз_продаж'].sum().reset_index()
axes[0, 0].bar(daily_forecast['Дата'], daily_forecast['Прогноз_продаж'], color='coral', alpha=0.7)
axes[0, 0].set_title('Прогноз продаж по дням Масленицы 2024', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Дата')
axes[0, 0].set_ylabel('Прогноз продаж, шт')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# 1.2. Прогноз продаж по торговым точкам
store_forecast = forecast_df.groupby('id_точки')['Прогноз_продаж'].sum().reset_index()
axes[0, 1].bar(store_forecast['id_точки'].astype(str), store_forecast['Прогноз_продаж'], color='steelblue', alpha=0.7)
axes[0, 1].set_title('Прогноз продаж по торговым точкам', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('ID торговой точки')
axes[0, 1].set_ylabel('Прогноз продаж, шт')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 1.3. Прогноз продаж по товарам
product_forecast = forecast_df.groupby('id_товара')['Прогноз_продаж'].sum().reset_index()
axes[1, 0].barh(product_forecast['id_товара'].astype(str), product_forecast['Прогноз_продаж'], color='mediumseagreen', alpha=0.7)
axes[1, 0].set_title('Прогноз продаж по товарам', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Прогноз продаж, шт')
axes[1, 0].set_ylabel('ID товара')
axes[1, 0].grid(True, alpha=0.3)

# 1.4. Топ-10 комбинаций точка-товар
top_combinations = forecast_df.groupby(['id_точки', 'id_товара'])['Прогноз_продаж'].sum().reset_index()
top_combinations['Комбинация'] = top_combinations['id_точки'].astype(str) + ' - ' + top_combinations['id_товара'].astype(str)
top_10 = top_combinations.nlargest(10, 'Прогноз_продаж')
axes[1, 1].barh(top_10['Комбинация'], top_10['Прогноз_продаж'], color='indianred', alpha=0.7)
axes[1, 1].set_title('Топ-10 комбинаций: Торговая точка - Товар', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Прогноз продаж, шт')
axes[1, 1].set_ylabel('Точка - Товар')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
try:
    plt.savefig('прогноз_продаж_масленица_2024.png', dpi=300, bbox_inches='tight')
except Exception as e:
    print(f"Предупреждение: не удалось сохранить график: {e}")
finally:
    plt.close()

# 2. Визуализация заказов
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 2.1. Объем заказа по дням
daily_orders = orders_df.groupby('Дата')['Объем_заказа'].sum().reset_index()
axes[0, 0].bar(daily_orders['Дата'], daily_orders['Объем_заказа'], color='darkorange', alpha=0.7)
axes[0, 0].set_title('Объем заказа по дням Масленицы 2024', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Дата')
axes[0, 0].set_ylabel('Объем заказа, шт')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# 2.2. Объем заказа по торговым точкам
store_orders = orders_df.groupby('id_точки')['Объем_заказа'].sum().reset_index()
axes[0, 1].bar(store_orders['id_точки'].astype(str), store_orders['Объем_заказа'], color='purple', alpha=0.7)
axes[0, 1].set_title('Объем заказа по торговым точкам', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('ID торговой точки')
axes[0, 1].set_ylabel('Объем заказа, шт')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 2.3. Сравнение прогноза и заказа
comparison = pd.merge(
    forecast_df.groupby('Дата')['Прогноз_продаж'].sum().reset_index(),
    orders_df.groupby('Дата')['Объем_заказа'].sum().reset_index(),
    on='Дата'
)
x = np.arange(len(comparison))
width = 0.35
axes[1, 0].bar(x - width/2, comparison['Прогноз_продаж'], width, label='Прогноз продаж', color='lightblue', alpha=0.7)
axes[1, 0].bar(x + width/2, comparison['Объем_заказа'], width, label='Объем заказа', color='lightcoral', alpha=0.7)
axes[1, 0].set_title('Сравнение прогноза продаж и объема заказа', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Дата')
axes[1, 0].set_ylabel('Количество, шт')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(comparison['Дата'].dt.strftime('%d.%m'), rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 2.4. Дефицит по дням
daily_deficit = orders_df.groupby('Дата')['Дефицит'].sum().reset_index()
axes[1, 1].bar(daily_deficit['Дата'], daily_deficit['Дефицит'], color='red', alpha=0.7)
axes[1, 1].set_title('Дефицит по дням (при текущем прогнозе)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Дата')
axes[1, 1].set_ylabel('Дефицит, шт')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
try:
    plt.savefig('объем_заказа_масленица_2024.png', dpi=300, bbox_inches='tight')
except Exception as e:
    print(f"Предупреждение: не удалось сохранить график: {e}")
finally:
    plt.close()

# 3. Детальная визуализация для одной торговой точки
sample_store = TARGET_STORES[0]
sample_data = forecast_df[forecast_df['id_точки'] == sample_store].copy()
sample_orders = orders_df[orders_df['id_точки'] == sample_store].copy()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Прогноз по товарам
pivot_forecast = sample_data.pivot_table(index='Дата', columns='id_товара', values='Прогноз_продаж', fill_value=0)
pivot_forecast.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab20')
axes[0].set_title(f'Прогноз продаж по товарам для точки {sample_store}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Прогноз продаж, шт')
axes[0].legend(title='ID товара', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Заказы по товарам
pivot_orders = sample_orders.pivot_table(index='Дата', columns='id_товара', values='Объем_заказа', fill_value=0)
pivot_orders.plot(kind='bar', stacked=True, ax=axes[1], colormap='tab20')
axes[1].set_title(f'Объем заказа по товарам для точки {sample_store}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('Объем заказа, шт')
axes[1].legend(title='ID товара', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
try:
    plt.savefig('детальный_прогноз_точка_{}.png'.format(sample_store), dpi=300, bbox_inches='tight')
except Exception as e:
    print(f"Предупреждение: не удалось сохранить график: {e}")
finally:
    plt.close()